In [1]:
import yaml
args = yaml.safe_load(open('config.yml'))
from transformers import GPT2Tokenizer

def load_tokenizer(args):
    tokenizer = GPT2Tokenizer.from_pretrained(args['model_name'])
    special_tokens = ['<speaker1>', '<speaker2>']
    tokenizer.add_special_tokens({
        'bos_token': '<bos>',
        'additional_special_tokens': special_tokens
    })

    # add new token ids to args
    special_tokens += ['<bos>', '<eos>']
    sp1_id, sp2_id, bos_id, eos_id = tokenizer.encode(special_tokens)
    args['sp1_id'] = sp1_id
    args['sp2_id'] = sp2_id
    args['bos_id'] = bos_id
    args['eos_id'] = eos_id

    return tokenizer
tokenizer = load_tokenizer(args)

/home/rahulbharti/Preojects/college-chatbot-gpt2/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# For Structured Tocken
from chatbot_files.utils import PadCollate
pad = PadCollate(args)
from chatbot_files.data import DialoguesDataset
train_dataset = DialoguesDataset('train', args)

test1 = train_dataset[0:5]
for data in test1[0]:
    # print(data)
    # break
    # print(len(data))
    print(len(data),tokenizer.decode(data))

100%|██████████| 27/27 [00:00<00:00, 25021.26it/s]

59 <bos> <speaker1> When was Rajkiya Engineering College (R. E. C.) Ambedkar Nagar established? <speaker2> Rajkiya Engineering College (R. E. C.) Ambedkar Nagar was established in 2010 by the Government of Uttar Pradesh under a special component plan.<|endoftext|>
88 <bos> <speaker2> Rajkiya Engineering College (R. E. C.) Ambedkar Nagar was established in 2010 by the Government of Uttar Pradesh under a special component plan. <speaker1> What were the initial branches offered by the college? <speaker2> The college initially offered B. Tech. Programs in three branches: Information Technology (IT), Electrical Engineering (EE), And Civil Engineering (CE), With an intake of 60 students in each branch.<|endoftext|>
70 <bos> <speaker1> How long did it take for the college to shift to its own campus? <speaker2> The college initially operated from the campus of Kamla Nehru Institute of Technology (K. N. I.T) in Sultanpur. It shifted to its own campus in Ambedkar Nagar in August 2012, About two 

In [3]:
# For corpus Data
from chatbot_files.utils import PadCollateCorpus
pad = PadCollateCorpus(args)
from chatbot_files.data import CorpusDataSet
train_dataset = CorpusDataSet('train', args)

test1 = train_dataset[0:5]
for data in test1[0]:
    # print(data)
    # break
    # print(len(data))
    print(len(data),tokenizer.decode(data))

77 Rajkiya Engineering College (R. E. C.) Ambedkar Nagar was established by Government of Uttar Pradesh under special component plan in year 2010, The college has started offering B. Tech Programme in three disciplines – Information Technology (IT), Electrical Engineering (EE) and Civil Engineering (CE) with intake of 60 seats in each branches from the session 2010-11.
105 Rajkiya Engineering College, Ambedkar Nagar is one of the best and most reputable government engineering colleges in the state of Uttar Pradesh. Rec Ambedkar Nagar has always been excelling both on the academic and the non-academic fronts. Rajkiya Engineering College, Ambedkar Nagar is an AICTE-approved government engineering college with a well-established library and labs. Every year a large number of students from the college clear the GATE exam and get placed into PSUs.
57 The work of college construction started in October 2010 with a budget of 6213. 81Lakh. The college was running in the campus of Kamla Nehru I

In [8]:
from torch.utils.data import DataLoader

train_loader =  DataLoader( train_dataset,
                            collate_fn=pad,
                            shuffle=True,
                            batch_size=args['batch_size'],
                            num_workers=1,
                            pin_memory=True)

In [9]:
from tqdm.auto import tqdm
for i, batch in enumerate(tqdm(train_loader)):
    print(len(batch))
    print(batch)
    break

  0%|          | 0/6 [00:00<?, ?it/s]

3
(tensor([[50257, 50258,  2061,   318,   262,  5761,   286,   262,  7511, 14044,
          5011,    30, 50259,   464,  5011, 17365,  3279,  4955,  3081,  3707,
           290,  4678,   284,  1826,   262,  6459,   287,   262,  2214,   286,
          3026,  8705,   290,  8676,   284,  3277,  2615,    13, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256],
        [50257, 50258,  2061,   389,   262, 24751,   286,  1770,    13, 43305,
           437,    84, 39523,   430,    30, 50259,  5246,    13, 43305,   437,
            84, 39523,   430,  6622,   257,   347,    13,  9634,   290,    44,
    

In [22]:
for i, batch in enumerate(tqdm(train_loader)):
    print(len(batch))
    input_ids, token_type_ids, labels = batch
    print(tokenizer.decode(input_ids[0]))
    print(input_ids[0])
    print(labels[0])
    break

  0%|          | 0/6 [00:00<?, ?it/s]

3
<bos> <speaker1> What are the areas of focus within the Electrical Engineering department at REC Ambedkar Nagar? <speaker2> The department focuses on providing a strong foundation in various areas of electrical engineering, Including communication technology, Control technology, Electronics, And power & energy.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>
tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,